# ID3

ID3 is a simple algorithm to create decision trees. It has several shortcomings, but still this is a good algorithm to start with.

## Metrics

### Entropy

$H(S) = \sum\limits_{x \in X} p(x) * \log_2(\frac1{p(x)})$

## Weather dataset (draft)

This is a pretty typical toy dataset for decision trees, but we should probably create more datapoints. Labels are also still missing.

In [86]:
Xw = np.array([
    ["sunny", "windy"],
    ["sunny", "not windy"],
    ["rainy", "not windy"]
])

## ID3 implementation

In [466]:
import numpy as np
from collections import defaultdict, Counter
from math import log as logarithm

class ID3:
    def __init__(self, max_depth=float("inf"), min_gain=0, depth=0):
        """
        Arguments:
            max_depth: After eaching this depth, the current node is turned into a leaf which predicts
                the most common label. This limits the capacity of the classifier and helps combat overfitting
            min_gain: The minimum gain a split has to yield. Again, this helps overfitting
            depth: Let's the current node know how deep it is into the tree, users usually don't need to set this
        """
        
        self.depth = depth
        self.max_depth = max_depth
        self.min_gain = min_gain
        
        # ID3 nodes are either nodes that make a decision or leafs which constantly predict the same result
        # We represent both possibilities using `ID3` objects and set `self.leaf` respectively
        self.leaf = False
        self.value = None
        
        self.children = {}
        self.feature = 0
    
    def fit(self, X, y):
        """
        Creates a tree structure based on the passed data
        
        Arguments:
            X: numpy array that contains the features in its rows
            y: numpy array that contains the respective labels
        """
        
        self.most_common_label = max(set(y), key=list(y).count)
        
        # If there is only one class left, turn this node into a leaf
        # and always return this one value
        if len(set(y)) == 1:
            self.leaf = True
            self.value = y[0]
        # If the tree is getting to deep, turn this node into a leaf
        # and always predict the most common value
        elif self.depth >= self.max_depth:
            self.leaf = True
            self.value = self.most_common_label
        # Otherwise, look for the most informative feature and do a split on its possible values
        else:
            self.feature = self._choose_feature(X, y)
            
            # If no feature is informative enough, turn this node into a leaf
            # and always predict the most common value
            if self.feature is None:
                self.leaf = True
                self.value = self.most_common_label
            else:
                for value, (Xi, yi) in self._partition(X, y, self.feature).iteritems():
                    child = ID3(depth=self.depth+1)
                    child.fit(Xi, yi)
                    self.children[value] = child
    
    def predict_single(self, x):
        """
        Predict the class of a single data point x by either using the value encoded in a leaf
        or by following the tree structure recursively until a leaf is reached
        
        Arguments:
            x: individual data point
        """
        
        if self.leaf:
            return self.value
        else:
            value = x[self.feature]
            
            if value in self.children:
                return self.children[value].predict_single(x)
            else:
                return self.most_common_label
        
    def predict(self, X):
        """
        Predict the results for an entire dataset
        
        Arguments:
            X: numpy array that contains each data point in a row
        """
        
        return [self.predict_single(x) for x in X]
    
    def score(self, X, y):
        """
        Returns the accuracy for predicting the given dataset X
        """
        
        correct = sum(self.predict(X) == y)
        return float(correct) / len(y)
        
    def _choose_feature(self, X, y):
        """
        Finds the most informative feature to split on and returns its index.
        If no feature is informative enough, `None` is returned
        """
        
        best_feature = 0
        best_feature_gain = -float("inf")
        
        for i in range(X.shape[1]):
            gain = self._information_gain(X, y, i)
            print "feature %d has gain %.2f" % (i, gain)
                        
            if gain > best_feature_gain:
                best_feature = i
                best_feature_gain = gain
        
        if best_feature_gain > self.min_gain:
            return best_feature
        else:
            return None
    
    def _information_gain(self, X, y, feature):
        """
        Calculates the information gain achieved by splitting on the given feature
        """
        
        result = self._entropy(y)
        
        summed = 0
        
        for value, (Xi, yi) in self._partition(X, y, feature).iteritems():
            summed += float(len(yi)) / len(y) * self._entropy(yi)
        
        result -= summed
        
        return result
    
    def _entropy(self, X):
        """
        Calculates the Shannon entropy on the given data X
        
        Arguments:
            X: An iterable for feature values. Usually, this is now a 1D list
        """
        
        summed = 0
        counter = Counter(X)

        for value in counter:
            count = counter[value]
            px = count / float(len(X))
            summed += px * logarithm(1. / px, 2)
        
        return summed
    
    def _partition(self, X, y, feature):
        """
        Partitioning is a common operation needed for decision trees (or search trees).
        Here, a partitioning is represented by a dictionary. The keys are values that the feature
        can take. Under each key, we save a tuple (Xi, yi) that represents all data points (and their labels)
        that have the respective value in the specified feature.
        """
        
        partition = defaultdict(lambda: ([], []))
        
        for Xi, yi in zip(X, y):
            bucket = Xi[feature]
            partition[bucket][0].append(Xi)
            partition[bucket][1].append(yi)
        
        partition = dict(partition)
            
        for feature, (Xi, yi) in partition.iteritems():
            partition[feature] = (np.array(Xi), np.array(yi))
            
        return partition

## xor dataset

`xor` is an easy test case: It's not linearly separable but it's clear what the output should be.

In [495]:
X = np.array([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1]
])

y = np.array([0, 1, 1, 0])

We have to specify `min_gain < 0` because we need to take non informative splits in the beginning to get a better split later.

In [496]:
clf = ID3(min_gain=-1)
clf.fit(X, y)

feature 0 has gain 0.00
feature 1 has gain 0.00
feature 0 has gain 0.00
feature 1 has gain 1.00
feature 0 has gain 0.00
feature 1 has gain 1.00


As expected, it works perfectly:

In [497]:
clf.score(X, y)

1.0

In [498]:
for Xi, yi in zip(X, y):
    print "Predicting", Xi, "as", clf.predict_single(Xi), "(correct = %d)" % yi

Predicting [0 0] as 0 (correct = 0)
Predicting [0 1] as 1 (correct = 1)
Predicting [1 0] as 1 (correct = 1)
Predicting [1 1] as 0 (correct = 0)


## Titanic

In [499]:
import pandas as pd
import numpy as np
from pandas import DataFrame
from sklearn.preprocessing import Imputer

def preprocess(data):    
    X = data.drop(["Survived", "Name", "Ticket", "Cabin", "Age", "Fare"], 1)
    X = pd.get_dummies(X)
    
    print X.head()
    
    imputer = Imputer()
    X = imputer.fit_transform(X.as_matrix())
    
    return X

In [500]:
data = DataFrame.from_csv("./titanic/train.csv")
y = data["Survived"].as_matrix()
X = preprocess(data)

             Pclass  SibSp  Parch  Sex_female  Sex_male  Embarked_C  \
PassengerId                                                           
1                 3      1      0         0.0       1.0         0.0   
2                 1      1      0         1.0       0.0         1.0   
3                 3      0      0         1.0       0.0         0.0   
4                 1      1      0         1.0       0.0         0.0   
5                 3      0      0         0.0       1.0         0.0   

             Embarked_Q  Embarked_S  
PassengerId                          
1                   0.0         1.0  
2                   0.0         0.0  
3                   0.0         1.0  
4                   0.0         1.0  
5                   0.0         1.0  


In [501]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

clf = ID3()
clf.fit(X_train, y_train)

feature 0 has gain 0.08
feature 1 has gain 0.03
feature 2 has gain 0.03
feature 3 has gain 0.22
feature 4 has gain 0.22
feature 5 has gain 0.02
feature 6 has gain 0.00
feature 7 has gain 0.01
feature 0 has gain 0.03
feature 1 has gain 0.02
feature 2 has gain 0.01
feature 3 has gain 0.00
feature 4 has gain 0.00
feature 5 has gain 0.01
feature 6 has gain 0.00
feature 7 has gain 0.00
feature 0 has gain 0.00
feature 1 has gain 0.02
feature 2 has gain 0.03
feature 3 has gain 0.00
feature 4 has gain 0.00
feature 5 has gain 0.01
feature 6 has gain 0.01
feature 7 has gain 0.00
feature 0 has gain 0.00
feature 1 has gain 0.01
feature 2 has gain 0.00
feature 3 has gain 0.00
feature 4 has gain 0.00
feature 5 has gain 0.01
feature 6 has gain 0.01
feature 7 has gain 0.01
feature 0 has gain 0.00
feature 1 has gain 0.01
feature 2 has gain 0.00
feature 3 has gain 0.00
feature 4 has gain 0.00
feature 5 has gain 0.00
feature 6 has gain 0.01
feature 7 has gain 0.01
feature 0 has gain 0.00
feature 1 has ga

In [502]:
print "train accuracy = %.5f" % clf.score(X_train, y_train)
print "test accuracy = %.5f" % clf.score(X_test, y_test)

train accuracy = 0.84270
test accuracy = 0.79330


Looking through the features being used:

In [486]:
child = clf
print clf.feature

while len(child.children) > 0:
    child = child.children[child.children.keys()[0]]
    print child.feature

3
0
2
5
6
1
None


## Comparison to sklearn

In [492]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

We get the exact same accuracy as sklearn!

In [503]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
print "train accuracy = %.5f" % clf.score(X_train, y_train)
print "test accuracy = %.5f" % clf.score(X_test, y_test)

train accuracy = 0.84270
test accuracy = 0.79330


Random forest doesn't seem to be very useful here. Probably because it's hard to overfit without having information about age or the paid price.

In [493]:
clf = RandomForestClassifier()
clf.fit(X_train, y_train)
print "train accuracy = %.5f" % clf.score(X_train, y_train)
print "test accuracy = %.5f" % clf.score(X_test, y_test)

train accuracy = 0.84270
test accuracy = 0.80447
